In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# ※ Quiz : 경주여행과 전주여행에 대해 최빈단어시각화와 유사도분석
- (1) naver open API를 활용하여 네이버지식인 "전주여행", "경주여행"를 각각 500건씩 검색 백업(naver.csv)
    * replace(), re.sum()이용
    * 백업 파일 결과 : query,no,title,link,description,total_text(title+' '+description)
- (2) 품사태깅 백업(naver_pos.csv)
    * query, no, token, pos
- (3) 명사만 추출 백업(naver_pos_nouns.csv)
    * query, token, pos
- (4) 빈도분석 백업(naver_pos_nouns_count.csv)
    * token, 경주빈도, 전주빈도, 빈도합
- (5) 빈도 시각화(워드클라우드, Text.plot)
    * 워드클라우드 이미지 저장
- (6) 단어간 유사도 분석(Word2Vec, 연관분석)

# 1. naver open API를 활용하여 검색어 500건 추출
- query,no,title,link,description,total_text(title+' '+description)

In [3]:
# .env 가져오기(방법1) - 추천
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [5]:
# .env 가져오기(방법2)
from decouple import config
# print(config('CLIENT_ID'))
# print(config('CLIENT_SECRET'))

In [8]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("경주 여행")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8')[:200])
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Tue, 02 Dec 2025 14:39:13 +0900",
	"total":1003198,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"내 마음대로 기록하는 <b>경주여행<\/b> 2박3일",
			"link":"https:\/\/blog.naver.com\/hello


In [11]:
# 방법1
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
import re  # 정규표현식
from html import unescape # description에 &lt;등의 특수문자를 <(문자)로 변경

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '경주 여행'
start = 1
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
# url = f'https://openapi.naver.com/v1/search/kin?query={query}&display=100&start={start}'
# response = requests.get(url, headers=headers)
url = 'https://openapi.naver.com/v1/search/kin'
params = {'query':query, 'display':100, 'start':start}
response = requests.get(url, params=params, headers=headers)
# print(response.text[:500])
# items = json.loads(response.text)['items']
items = response.json()['items']
print(len(items))

100

In [13]:
items[97]

{'title': '<b>경주 여행</b>지 추천',
 'link': 'https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90111&docId=261370795&qb=6rK97KO8IOyXrO2WiQ==&enc=utf8',
 'description': '... 2.부산을 벗어난 <b>경주 여행</b>지 알려주세요! 3.당일치기로... <b>경주 여행</b>코스 -----&gt; 밑에 자세하게 권역별로 여행코스를 소개했습니다. <b>경주여행</b>이 처음이라면 대릉원, 첨성대... 참고로, 쭈니아빠 블로그를 방문하시면, <b>경주여행</b>... '}